##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: JSON Mode Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb"><img src="../images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

The Gemini API can be used to generate a JSON output if you set the schema that you would like to use.

Two methods are available. You can either set the desired output in the prompt or supply a schema to the model separately.

### Install dependencies

In [2]:
!pip install -U -q "google-generativeai>=0.7.2"

In [3]:
import google.generativeai as genai

import json
import dataclasses
import typing_extensions as typing

### Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [4]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Set your constrained output in the prompt

 Activate JSON mode by specifying `respose_mime_type` in the `generation_config` parameter:

In [5]:
model = genai.GenerativeModel("gemini-2.0-flash",
                              generation_config={"response_mime_type": "application/json"})

For this first example just describe the schema you want back in the prompt:

In [6]:
prompt = """List a few popular cookie recipes using this JSON schema:

Recipe = {'recipe_name': str}
Return: list[Recipe]"""

In [7]:
raw_response = model.generate_content(prompt)

Parse the string to JSON:

In [8]:
response = json.loads(raw_response.text)
print(response)

[{'recipe_name': 'Chocolate Chip Cookies'}, {'recipe_name': 'Peanut Butter Cookies'}, {'recipe_name': 'Sugar Cookies'}, {'recipe_name': 'Oatmeal Raisin Cookies'}, {'recipe_name': 'Snickerdoodles'}]


For readability serialize and print it:

In [9]:
print(json.dumps(response, indent=4))

[
    {
        "recipe_name": "Chocolate Chip Cookies"
    },
    {
        "recipe_name": "Peanut Butter Cookies"
    },
    {
        "recipe_name": "Sugar Cookies"
    },
    {
        "recipe_name": "Oatmeal Raisin Cookies"
    },
    {
        "recipe_name": "Snickerdoodles"
    }
]


## Supply the schema to the model directly

The newest models (1.5 and beyond) allow you to pass a schema object (or a python type equivalent) directly and the output will strictly follow that schema.

Following the same example as the previous section, here's that recipe type:

In [10]:
class Recipe(typing.TypedDict):
    recipe_name: str
    recipe_description: str
    recipe_ingredients: list[str]

For this example you want a list of `Recipe` objects, so pass `list[Recipe]` to the `response_schema` field of the `generation_config`.

In [11]:
model = genai.GenerativeModel(model_name="models/gemini-2.0-flash")

result = model.generate_content(
    "List a few imaginative cookie recipes along with a one-sentence description as if you were a gourmet restaurant and their main ingredients",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema = list[Recipe]),
    request_options={"timeout": 600},
)

In [12]:
print(json.dumps(json.loads(result.text), indent=4))

[
    {
        "recipe_name": "Celestial Cranberry Nibblers\", these cookies combine tart cranberries with crunchy cacao nibs, creating a delightful sweet and bitter symphony with a celestial flair and offer a harmonious blend of contrasting tastes and textures that will leave you craving more.\n\nIngredients:\n\n- All-purpose flour\n- Dried cranberries\n- Cacao nibs\n- Unsalted butter\n- Granulated sugar\n- Eggs\n- Vanilla extract\n- Baking powder\n- Salt\n\nInstructions:\n\n1. Preheat your oven to 350\u00b0F (175\u00b0C). Line a baking sheet with parchment paper.\n2. In a bowl, whisk together the flour, baking powder, and salt.\n3. In a separate bowl, cream together the butter and sugar until light and fluffy.\n4. Beat in the egg and vanilla extract.\n5. Gradually add the dry ingredients to the wet ingredients, mixing until just combined.\n6. Fold in the cranberries and cacao nibs.\n7. Drop by rounded tablespoons onto the prepared baking sheet.\n8. Bake for 10-12 minutes, or until t

It is the recommended method if you're using a compatible model.

## Next Steps
### Useful API references:

Check the [structured ouput](https://ai.google.dev/gemini-api/docs/structured-output) documentation or the [`GenerationConfig`](https://ai.google.dev/api/generate-content#generationconfig) API reference for more details

### Related examples

* The constrained output is used in the [Text summarization](../examples/json_capabilities/Text_Summarization.ipynb) example to provide the model a format to summarize a story (genre, characters, etc...)
* The [Object detection](../examples/Object_detection.ipynb) examples are using the JSON constrained output to uniiformize the output of the detection.

### Continue your discovery of the Gemini API

JSON is not the only way to constrain the output of the model, you can also use an [Enum](../quickstarts/Enum.ipynb). [Function calling](../quickstarts/Function_calling.ipynb) and [Code execution](../quickstarts/Code_Execution.ipynb) are other ways to enhance your model by either using your own functions or by letting the model write and run them.